In [1]:
class edge:
      def __init__(self):
        self.label=None
        self.destination=None
      def __str__(self):
        return f"edge {self.label} going to {self.destination}"

In [5]:
class NFA_state:
  def __init__(self):
    self.label=None
    self.out_edges=[]


  '''
  This function gets the direct neighbours from taking the Epsilon edge.
  just the direct neighbours.
  '''
  def get_epsilon_closure(self):
    closure = set()
    closure.add(self)
    for edge in self.out_edges:
      if edge.label == "ε":
        closure.add(edge.destination)
    return closure
  
  '''
    gets the direct neighbours from taking the Character input edge, just the direct neighbours.
  '''
  def get_char_closure(self, char):
    closure = set()
    for edge in self.out_edges:
      if edge.label == char:
        closure.add(edge.destination)
    return closure
  
  '''
    Returns a set of States reachable from taking Char edges recursively
  '''
  def get_reachable_states(self, char):
    reachable_states = set()
    for edge in self.out_edges:
      if edge.label == char:
        reachable_states.add(edge.destination)
        reachable_states = reachable_states.union(edge.destination.get_reachable_states(char))
    return reachable_states
  
  '''
    Returns a set of States reachable from taking Epsilon edges recursively
  '''
  def get_reachable_states_epsilon(self):
    reachable_states = set()
    for edge in self.out_edges:
      if edge.label == "ε":
        reachable_states.add(edge.destination)
        reachable_states = reachable_states.union(edge.destination.get_reachable_states_epsilon())
    return reachable_states
  
  

  def __repr__(self) -> str:
     return self.label
  def __eq__(self, other):
    return other.label == self.label
  def __ne__(self, other):
    return other.label != self.label
  def __hash__(self):
     return hash(self.label)

In [6]:
class NFA:

    def __init__(self, initial, acc, inner):
        self.start = initial
        self.accept = acc
        self.inner_states = inner

    
    '''
        takes a SuperState and an input Character, 
        gets the Char closure from all the substates in the SuperState,
        then makes a new list of reachable states (as a candidate new SuperState), returns this list
    '''
    def generate_new_superstate(self, super_state, char):
        reachable_states = set()
        for state in super_state:
            reachable_states = reachable_states.union(state.get_char_closure(char))

        new_superstate = set()
        for state in reachable_states:
            new_superstate = new_superstate.union(state.get_epsilon_closure())


        return new_superstate

    def __repr__(self) ->str:
      return self.start.label


In [7]:
class DFA_Edge:
    def __init__(self):
      self.label = None
      self.destination = None

In [8]:
class SuperState:
    '''A superState is defined by a bunch of SubStates, a bunch of outgoing edges, and its label/ID,
      and 2 flags indicating whether or not this SuperState is starting or accepting
    '''
    def __init__(self):
        self.is_start = None          
        self.is_end = None            
        self.out_edges = []        
        self.sub_states = set()        
        self.label = None

    def get_labels(self):
      labels = []
      for s in self.sub_states:
        labels.append(s.label)
      return labels

    def __eq__(self, other):
      set_of_src_labels = set()
      set_of_dst_labels = set()
      for s in self.sub_states:
        set_of_src_labels.add(s.label)
      for s in other.sub_states:
        set_of_dst_labels.add(s.label)
      return set_of_src_labels == set_of_dst_labels
    
    def __ne__(self, other):
      set_of_src_labels = set()
      set_of_dst_labels = set()
      for s in self.sub_states:
        set_of_src_labels.add(s.label)
      for s in other.sub_states:
        set_of_dst_labels.add(s.label)
      return set_of_src_labels != set_of_dst_labels

    def __str__(self):
      if(self.sub_states):
        stringy = ""
        for sub in self.sub_states:
          stringy = stringy + str(sub) + ","
        return "SuperState " +stringy
      else:
        return "Empty SuperState"

    def __repr__(self):
      if(self.sub_states):
        stringy = ""
        for sub in self.sub_states:
          stringy = stringy + str(sub) + ","
        return stringy
      else:
        return "Empty SuperState"

    def __hash__(self):
      if(self.sub_states):
        stringy = " "
        for sub in self.sub_states:
          stringy = stringy + str(sub) + ","
        return hash("SuperState" +stringy)
      else:
        return hash(None)

In [9]:
class DFA:
    ''' A DFA object contains a set of SuperStates, also a pointer to the starting SuperState and a set of the accept SuperState'''
    def __init__(self):
        self.start_super_state = None          
        self.end_super_states = set()              
        self.super_states = set() 

    '''Just makes a new empty DFA, and a legit SuperState then insert this initial SuperState into the Empty DFA then return the DFA'''
    def __init__(self, nfa: NFA):
      self.start_super_state = SuperState()
      self.start_super_state.is_start = True
      self.start_super_state.label = "S0"
      
      self.start_super_state.sub_states = nfa.start.get_epsilon_closure()
      self.start_super_state.out_edges = []
      self.super_states.add(self.start_super_state)
      self.end_super_states = set()

      
    '''takes a list of states, returns the matching SuperState if the list already'''
    def get_super_state(self, states):
      for super_state in self.super_states:
        if super_state.sub_states == states:
          return super_state
      return None

    def __eq__(self, other):
      return self.start_super_state == other.start_super_state and self.super_states == self.super_states
    
    def __ne__(self, other):
      return self.start_super_state != other.start_super_state or self.super_states != self.super_states

    
    def empt(self):
      self.start_super_state = SuperState()          
      self.end_super_state = set()             
      self.super_states = set()             